In [36]:
!pip install plotly-calplot pandas --quiet

In [37]:
import pandas as pd  # pip install pandas
from plotly_calplot import calplot

In [52]:
df = pd.read_csv("fichier_sport_csv.csv", encoding='latin-1')
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')


In [136]:
df

,Date,Exercise,Details,Sensation,Sensation_clean
0,2023-09-29,10min rowing,NaN,5/5,5.0
1,2023-09-29,Tirage vertical,49kg,5/5,5.0
2,2023-09-29,DIPS,49kg,5/5,5.0
3,2023-09-29,Shoulder press,15kg1 10kg3,5/5,5.0
4,2023-09-29,Leg press,40kg1 50kg3,5/5,5.0
...,...,...,...,...,...
1204,2025-01-24,Tirage vertical large,35kg4,3/5,3.0
1205,2025-01-24,Dips,35kg4,3/5,3.0
1206,2025-01-24,Souleve de terre roumain,10kg2 20kg3,3/5,3.0
1207,2025-01-24,Mollet barre guide,40kg4,3/5,3.0


In [53]:
df["Sensation_clean"] = df["Sensation"].str.replace("/5", "").astype(float)

In [54]:
df1 = df[["Date","Sensation_clean"]].drop_duplicates().dropna()

In [58]:
df1['DayOfWeek'] = df1["Date"].dt.day_name()

In [59]:
df1

,Date,Sensation_clean,DayOfWeek
0,2023-09-29,5.0,Friday
8,2023-10-04,4.0,Wednesday
16,2023-10-06,5.0,Friday
24,2023-10-11,3.0,Wednesday
32,2023-10-13,3.0,Friday
...,...,...,...
1172,2024-01-15,4.0,Monday
1178,2025-01-17,5.0,Friday
1188,2025-01-19,5.0,Sunday
1194,2025-01-22,4.0,Wednesday


In [51]:
custom_colorscale = [
    [0, 'red'],      # Value 1 mapped to red
    # [0.5, 'yellow'], # Intermediate color (you can adjust this)
    [1, 'green']    # Value 5 mapped to green
]


# Create a simple calendar heatmap
fig = calplot(
    df1,
    x="Date",
    y="Sensation_clean",
    gap=1,
    years_title=True,
    # month_lines_width=1.5,
    # month_lines_color="black"
    # dark_theme=True
    colorscale= "bluyl"
    # colorscale=custom_colorscale
)

fig.show()
fig.write_html("calendar_heatmap.html")

In [95]:
import plotly.express as px

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']



# Calculate the frequency of each day
df1['DayOfWeek_ordered'] = pd.Categorical(df1['DayOfWeek'], categories=day_order, ordered=True)

day_counts = df1['DayOfWeek_ordered'].value_counts().sort_index()



# Create the horizontal bar plot
fig = px.bar(
    x=day_counts.values,
    y=day_counts.index,
    orientation='h',  # Make it horizontal
    category_orders={'DayOfWeek': day_order}, # set the order for display
    # text_auto=True,
    # text_auto='.1s',
    text = day_counts.values,

)

fig.update_layout(
    title='Nb of trainings per Days of the Week',
    xaxis_title='Nb of trainings',
    yaxis_title='',#'Day of the Week',
    yaxis_tickangle=10,

)

# Customize the abrs
fig.update_traces(textfont_size=20, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(
    marker_color='rgb(158,202,225)',
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.8,
    marker=dict(cornerradius=30)
)

# Customize the text to display outside of each bar
fig.update_traces(texttemplate='%{text:.1s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [141]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots



day_order = ['Sunday', 'Saturday', 'Friday', 'Thursday', 'Wednesday', 'Tuesday', 'Monday']


# Calculate the frequency of each day
df1['DayOfWeek_ordered'] = pd.Categorical(df1['DayOfWeek'], categories=day_order, ordered=True)

day_counts = df1['DayOfWeek_ordered'].value_counts().sort_index()
day_average_sensation = df1["Sensation_clean"].groupby(df1["DayOfWeek_ordered"]).mean()


# Create a subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2)

# Add the first bar plot to the first subplot (column 1)
fig.add_trace(
    go.Bar(
        x=day_counts.values,
        y=day_counts.index,
        orientation='h',
        text = day_counts.values,
        marker_color='rgb(158,202,225)',
        marker_line_color='rgb(8,48,107)',
        marker_line_width=1.5,
        opacity=0.8,
        marker=dict(cornerradius=30),
        textfont_size=20,
        textangle=0,
        textposition="outside",
        cliponaxis=False,
        texttemplate='%{text:.2s}'
    ),
    row=1, col=1
)

# Add the duplicate bar plot to the second subplot (column 2)
fig.add_trace(
    go.Bar(
        x=day_average_sensation.values,
        y=day_average_sensation.index,
        orientation='h',
        text = day_average_sensation.values,
        marker_color='lavender',
        marker_line_color='rgb(8,48,107)',
        marker_line_width=1.5,
        opacity=0.8,
        marker=dict(cornerradius=30),
        textfont_size=20,
        textangle=0,
        textposition="outside",
        cliponaxis=False,
        texttemplate='%{text:.3s}/5'
    ),
    row=1, col=2
)

# Update layout for both subplots
fig.update_layout(
    title_text="",
    title_x=0.5,  # Set the title's horizontal position to the center (0.5)
    title_y=0.95,  # Set the title's vertical position near the t
    title_font=dict(size=24),  # Set the title font size
    xaxis_title='Nb of training sessions for each day of the week',
    yaxis_title='',
    xaxis2_title='Average rated sensation out of 5 for each day of the week',  # Title for x-axis of the second subplot
    yaxis2_title='',  # Title for y-axis of the second subplot
    showlegend=False #hide the legend
    # legend=dict(
    #     x=0.5,  # Adjust x position (0 to 1)
    #     y=1.1,   # Adjust y position (0 to 1)
    #     xanchor='center',  # Horizontal alignment ('auto', 'left', 'center', 'right')
    #     yanchor='bottom',  # Vertical alignment ('auto', 'top', 'middle', 'bottom')
    #     orientation="h",  # Orientation ('v', 'h')
    #     font=dict(size=12),  # Font size
    #     bgcolor='rgba(255, 255, 255, 0.5)'  # Background color with transparency
    # )
)

# Add a vertical line at x = 4
fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=df1["Sensation_clean"].mean(),
        x1=df1["Sensation_clean"].mean(),
        y0=0,
        y1=1,
        xref="x2",  # Refer to the x-axis of the second subplot
        yref="paper",  # Refer to the entire plot area for y-axis
        line=dict(color="red", width=2, dash="dash")
    )
)


# Show the plot
fig.show()

<ipython-input-141-325b407d050a>:13: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [142]:
df1["Sensation_clean"].mean()

3.66875